In [1]:
import random

# Modules to import

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores.faiss import FAISS

# Get OpenAI API KEY

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file
api = os.getenv('OPENAI_API_KEY_H')

# Split Chunk

In [3]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

docs = []
for loader in os.listdir("docs"):
    if loader is not None :
        try :
            docs.extend(PyPDFLoader("docs/"+loader).load())
        except Exception as e1 :
            print(f" [ Documents ERROR ] : {e1}")
    else : break

In [4]:
docs[:2]

[Document(page_content='1-72 สารบัญ ภาพโดยรวม/คู่มืออย่างง่าย        1 ข้อมูลทั่วไป          2 การล็อกและการปลดล็อก        3 เบาะนั่งและเข็มขัดนิรภัย         4 แผงหน้าป้ดและอุปกรณ์ควบคุม        5 การสตาร์ทและการขับขี่         6 การใช้อุปกรณ์อํานวยความสะดวก       7 เมื่อเกิดเหตุฉุกเฉิน         8 การดูแลรักษาสภาพรถ         9 การบํารุงรักษา          10 ข้อมูลจําเพาะ          11 ', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 0}),
 Document(page_content='แผงหน้าป้ดและอุปกรณ์ควบคุม(ด้านคนขับ) ภาพโดยรวม/คู่มืออย่างง่าย1-11 - สวิตช์ที่ป้ดนํ้าฝนและฉีดนํ้าล้างกระจกหน้า5-45 - แผงหน้าป้ดหน้า5-2 - สวิตช์ควบคุมความสว่างของแผงหน้าป้ดหน้า5-2 - สวิตช์ระบบควบคุมเสถียรภาพการทรงตัว(ASC)OFFหน้า6-70 - สวิตช์เซ็นทรัลล็อกหน้า3-15 - สวิตช์ควบคุมหน้าต่างไฟฟ้าหน้า3-19 - สวิตช์ล็อกหน้า3-20 - สวิตช์ไฟหน้าและไฟสูง-ตํ่าหน้า5-40 - สวิตช์ระบบปรับไฟสูงอัตโนมัติ(AHB)หน้า5-43 - คันสวิตช์ไฟเลี้ยวหน้า5-44 - สวิตช์ไฟตัดหมอกด้านหน้าหน้า5-45 - สวิตช์หน้าจอแสดงข้อมูลรวมหน้า5-4 - สวิตช์ควบคุมเครื่องเสียงบนพวงมาลัยให้ดูคู่

# Vector Database

In [5]:
splits = r_splitter.split_documents(docs)

db = FAISS.from_documents( splits, OpenAIEmbeddings( api_key = api ) )

In [6]:
splits[random.randint(0,len(splits))]

Document(page_content=': กะพริบ (ตัวแสดงโหมดการขับเคลื่อน4 ล้อกะพริบด้วย) : สว่าง : ดับ “4H”  “4HLc” \n: กะพริบ (ตัวแสดงโหมดการขับเคลื่อน 4 ล้อกะพริบด้วย) : สว่าง : ดับ ตําแหน่งปุ่มเลือกการขับเคลื่อน 4 ล้อตัวแสดง2Hกําลังเปลี่ยนโหมดการขับเคลื่อน 4 ล้อ4H\nตําแหน่งปุ่มเลือกการขับเคลื่อน 4 ล้อตัวแสดง4Hกําลังเปลี่ยนโหมดการขับเคลื่อน 4 ล้อ4HLc', metadata={'source': 'docs/Text_PDF_.pdf', 'page': 196})

# CHAT

In [7]:
# Memory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer',
)

In [8]:
model_lists = [ "gpt-3.5-turbo", 'gpt-4-1106-preview', "gpt-4", ]

llm = ChatOpenAI( api_key = api, model_name = model_lists[0], temperature = 0 )

In [34]:
# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
#     chain_type="map_reduce",
#     return_source_documents = True,
# )

In [12]:
memory.clear()

In [9]:
qa = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
            memory = memory,
            return_source_documents = True,
            return_generated_question = True,
        )

## Get answer (Testing)

In [14]:
ran = splits[random.randint(0,len(splits))]

prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""

# Test
result = qa({"question": prompt})

print(f"context : {(ran.page_content)[:100]}\npage : {(ran.metadata)['page']+1}\n\n")

print(f"{result.keys()}\n\n")
print(f"Question :\n{result['generated_question']}\n\nAnswer :\n{result['answer']}\n\nSource :\nPage : {(result['source_documents'])[0].metadata}\nContent : {(result['source_documents'])[0].page_content}")

context : หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก
page : 210


dict_keys(['question', 'chat_history', 'answer', 'source_documents', 'generated_question'])


Question :

context : 'หมายเหตุ  
● หากคุณยังคงกดสวิตช์  “ASC OFF” ค้างไว้หลังจากที่ฟังก์ชันควบคุมเสถียรภาพปิดไปแล้ว  “ฟังก์ชันป้องกันการทํางานผิดพลาด ” 
จะทํางานและเปิดฟังก์ชันควบคุมเสถียรภาพอีกครั้ง  
● แม้จะปิดฟังก์ชันควบคุมเสถียรภาพไปแล้ว  แต่ฟังก์ชันนี้อาจกลับมาทํางานได้โดยขึ้นอยู่กับการเคลื่อนไหวของรถ  
 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้ง  
E00617001048 
ฟังก์ชันควบคุมการออกตัวและการเข้าโค้งป้องกันไม่ให้ล้อหมุนมากเกินไปบนถนนลื่นและช่วยให้รถที่หยุดอยู่ออกตัวได้  
นอกจากนี้ยังให้กําลังการขับเคลื่อนที่เพียงพอและทําให้การหมุนพวงมาลัยมีประสิทธิภาพเมื่อรถเลี้ยวขณะเหยียบคันเร่ง 
 
ข้อควรระวัง'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate quest

In [15]:
((result['source_documents'])[0].metadata)['page']+1

210

In [14]:
# del result

# Prepare_dataset

In [5]:
import pandas as pd

df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [17]:
rerank_df = {"questions":[], "answer":[], "doc_page":[], "doc_content":[], "doc_source":[]}

In [18]:
for i in range(51):
    ran = splits[random.randint(0,len(splits))]
    prompt = f"""
context : '{ran.page_content}'
Create 1 question using only the context provided.
End each question with a '?' character.
You must answer the question by refer to the context as much as you can.
Separate question/answer by a newline.
the question and the answer should be thai language.
"""
    try :
        result = qa({"question": prompt})
        
        rerank_df["answer"].append(result['answer'])
        rerank_df["questions"].append(result['generated_question'])
        rerank_df["doc_content"].append(ran.page_content)
        rerank_df["doc_page"].append( (ran.metadata)['page']+1 )
        rerank_df["doc_source"].append( (ran.metadata)['source'] )
        
    except :
        memory.clear()
        result = qa({"question": prompt})
 

In [19]:
r_df = pd.DataFrame.from_dict(rerank_df)

print(f"number of dataset : {len(r_df)}\n\n")
r_df.tail(3)

number of dataset : 42




,questions,answer,doc_page,doc_content,doc_source
39,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
40,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
41,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [20]:
df = pd.concat([df, r_df], 
                  ignore_index = True)

print(f"number of dataset : {len(df)}\n\n")
display(df.tail(3))

number of dataset : 93




,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [32]:
df.to_csv("rerank_dataset.csv")

# Text Labeling

In [2]:
from openai import OpenAI

client = OpenAI(api_key=api)

In [26]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [36]:
categories = {
            'parts':[ 'ประตู', 'หน้าต่าง', 'กระจกมองข้าง', 'กระจกหน้า หลัง', 'ล้อ', 'ยาง', 'กระโปรง', 'เกียร์', 'สวิตช์', 'หน้าปัด', 'ระบบ',
                    'ถังน้ำมัน', 'กุญแจ', 'เบรก', 'กล้อง', 'หน้าจอ', 'เฟือง', 'แอร์ เครื่องปรับอากาศ', 'เบาะ', 'ช่องเสียบ พอร์ต', 'ที่วาง',
                     'ถุงลมนิรภัย', 'แตร', 'รีโมท', 'ช่องเก็บ', 'ไฟฟ้า', 'ไฟ', 'เข็มขัดนิรภัย', 'เซ็นเซอร์',],

            'events':[ 'ห้าม อย่า', 'เติม', 'รอย', 'เสียหาย', 'แจ้งเตือน', 'มืด สว่าง', 'รบกวน', 'ชน', 'เตือน ระวัง', 'ลากรถ', 'เปลี่ยน',
                      'ลม', 'ไฟหรี่', 'เร็ว ช้า', 'สูง ต่ำ', 'เช็ด ล้าง ทำความสะอาด', 'ฝน', 'ถนน',]
              }

In [27]:
# test
num = 3

prompt = f"""
context : '{num}'
Your task is to label the context provided.
if there're more than 1 label, 
summarize them into 1 keyword in thai language.
"""

lang = get_completion(prompt)
print(f"Answer : {lang}\n\nOriginal : {df['doc_content'][num]}")

Answer : ความรู้ (knowledge)

Original : เป็นเหตุให้เสียงเตือนดังและไฟเตือนสว่างขึ้นได้ -  [รถแบบที่ 1]สําหรับรถรุ่นที่ติดตั้งการเตือนคาดเข็มขัดนิรภัยสําหรับเบาะนั่งผู้โดยสารด้านหลัง เบาะนั่งที่ไม่ได้คาดเข็มขัดนิรภัยสําหรับผู้โดยสารด้านหลังจะถูกรีเซ็ตหลังจากเปิดประตูหลังบานใดบานหนึ่งในขณะที่รถหยุดและปิดประตูทุกบาน สําหรับเบาะนั่งผู้โดยสารด้านหลัง* 1- เบาะนั่งสีแดงที่มีสัญลักษณ์ “X”: แสดงว่าเข็มขัดนิรภัยที่เกี่ยวข้องไม่ได้คาด 2- เบาะนั่งสีเขียวที่มีเครื่องหมายถูก: คาดเข็มขัดนิรภัยที่เกี่ยวข้อง การเตือนคาดเข็มขัดนิรภัยด้านหลังจะแสดงบนหน้าจอแสดงข้อมูลรวมเป็นเวลาประมาณ 65 วินาทีเมื่อโหมดการทํางานอยู่ที่ ON


In [8]:
type(lang)

str

In [12]:
temp_df = {'label':[], 'questions':[]}

In [13]:
for n,num in enumerate(df['doc_content']):
    prompt = f"""
context : '{num}'
Your task is to label the context provided.
if there're more than 1 label, 
summarize them into 1 keyword in thai language.
"""

    lang = get_completion(prompt)
    temp_df['questions'].append(df['questions'][n])
    temp_df['label'].append(lang)
    

In [28]:
print(f"Number of label {len(temp_df['label'])}\nNumber of questions : {len(temp_df['questions'])}")

Number of label 93
Number of questions : 93


In [30]:
temp_df = pd.DataFrame.from_dict(temp_df)
df = pd.merge(left=df, right=temp_df, how='left', on='questions')

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source,label
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf,อุปกรณ์ควบคุม
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf,ระบบ FCM
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf,โหมดการขับเคลื่อน 4 ล้อ


In [34]:
len(df.label.unique())

86

In [35]:
df.label.unique()

array(['สวิตช์ล็อก', 'การขับขี่ขึ้นทางชัน',
       'อุปกรณ์ภายนอก-ด้านหน้า: ซิงเกิลแค็บ, เสาอากาศ, การควบคุมหน้าต่างไฟฟ้า, ฝาปิดถังน้ำมันเชื้อเพลิง, การล็อกและปลดล็อก, ระบบกุญแจรีโมท, กระจกมองข้าง, ไฟเลี้ยวด้านข้าง, ไฟเลี้ยวด้านหน้า, ไฟหน้า, ไฟสูงต่ำ, ไฟหรี่, ที่ปัดน้ำฝนและฉีดน้ำล้างกระจก, ฝากระโปรง, ห้องเครื่องยนต์',
       'เตือนคาดเข็มขัดนิรภัย', 'คาดการณ์รถ', 'การขับเคลื่อน 4 ล้อ',
       'บำรุงรักษา', 'ถุงลมนิรภัย', 'รถยนต์', 'ระบบเซ็นเซอร์จอดรถ',
       'ระบบช่วยเตือนขณะถอยรถ (RCTA)', 'ระบบ ASTC',
       'แผงหน้าปัด, อุปกรณ์ควบคุม, ไฟแสดงการทำงาน, ไฟเตือน, หน้าจอแสดงข้อมูล, แบตเตอรี่กุญแจอัจฉริยะ, กุญแจอัจฉริยะ, รถ, สวิตช์เครื่องยนต์, เสียงเตือน, แป้นคลัตช์, แป้นเบรก, เกียร์อัตโนมัติ, หากกุญแจอัจฉริยะทำงานไม่ถูกต้อง, เข็มขัดนิรภัย, เบาะนั่ง',
       'ASTC หรือ ASTC OFF', 'ป้องกัน',
       'การช่วยเหลือ, การตั้งค่า, นาฬิกา, การบำรุงรักษา, จอแสดง, หน่วยภาษา, การเชื่อมโยงกับกุญแจ, การรีเซ็ตเป็นค่าเริ่มต้น, หน้าจอแสดงข้อมูลรวม',
       'อุปกรณ์นิรภัยสำหรับเด็ก', 'ระบบถุงลมนิรภัย SRS'